In [1]:
import sys
sys.path+=['/Users/lunzhang/anaconda3/lib/python3.6/site-packages']
import DSGRN
import os
import json

In [2]:
from total2partial import *
import time 
import re
import json
import numpy as np
from itertools import product
from itertools import permutations
import os
import sys

In [3]:
nwStr = "x1:~x2 \n x2:~x3 \n x3:~x1"
network = DSGRN.Network(nwStr)
DSGRN.DrawGraph(network)
parameterGraph = DSGRN.ParameterGraph(network)
parameterGraph.size()

27

In [4]:
parameterSample = parameterGraph.parameter(0)
domainGraph = DSGRN.DomainGraph(parameterSample)
morseDecomposition = DSGRN.MorseDecomposition(domainGraph.digraph())
morseGraph = DSGRN.MorseGraph(domainGraph, morseDecomposition)

In [5]:
morseGraph.poset().size()
morseGraph.annotation(0)[0]

'FP { 0, 0, 0 }'

In [6]:
notOscillatory = []
size = parameterGraph.size()
for i in range(size):
    parameterSample = parameterGraph.parameter(i)
    domainGraph = DSGRN.DomainGraph(parameterSample)
    morseDecomposition = DSGRN.MorseDecomposition(domainGraph.digraph())
    morseGraph = DSGRN.MorseGraph(domainGraph, morseDecomposition)
    l = morseGraph.poset().size()
    notOscillatory.append(parameterSample)
    #print(j)
    for j in range(l):
        if morseGraph.annotation(j)[0].startswith('FC'):
            notOscillatory = notOscillatory[:-1]
            break        

In [7]:
len(notOscillatory)

26

In [8]:

# suppose there are at most 2 thresholds
def stringifyParser(l):
    ret =  []
    l = eval(l)
    #print(l)
    for i in range(len(l)):
        inputNo = l[i][1][0]  
        m=outputNo = l[i][1][1] 
        polyNo = pow(2,inputNo)
        totalLen = polyNo*outputNo

        logic = l[i][1][2]
        logic = bin(int(logic,16))[2:].zfill(totalLen)

        lower = []
        middle = []
        upper = []
        thetas = l[i][2]
        temp = []
        for j in range(0,len(logic),m):
            polyIndex = polyNo - int(j/m) - 1
            piece = logic[j:j+m]
            #print(piece)
            if len(set(piece)) == 2:
                middle.append(polyIndex)
            elif set(piece) == set('1'):
                upper.append(polyIndex)
            else:
                lower.append(polyIndex)

        if len(thetas)==1:
            temp = sorted(lower) + [-thetas[0]-1] + sorted(upper)
        else:
            
            temp = sorted(lower) + [-thetas[0]-1] + sorted(middle) + [-thetas[1]-1]+sorted(upper)
            #print(temp)

        ret.append(tuple(temp))
    return ret

In [9]:
pnlist = [stringifyParser(x.stringify()) for x in notOscillatory]

In [10]:
def loadAtomicChainComplex(fpath):
    ret = dict()
    with open(fpath,'r') as file:
        atomicCC = json.load(file)
    for key in atomicCC:
        newk = key[1:-1].split(',')
        newk = tuple([int(x) for x in newk])
        v = atomicCC[key]
        newvalue = dict()
        for dim in v:
            newfaces = set([tuple(x) for x in v[dim]])
            newvalue[int(dim)] = newfaces
        ret[newk] = newvalue 
    return ret

def integerPartitionHelper(Sum,UpperBounds,path,ret):
    if len(path) == len(UpperBounds):
        if Sum!=0:
            return
        else:
            ret.append(path.copy())
            return 
    
    for i in range(min(Sum,UpperBounds[len(path)])+1):
        path.append(i)
        integerPartitionHelper(Sum-i,UpperBounds,path,ret)
        #print(path)
        path.pop()
        
    return 
            

def integerPartition(Sum, UpperBounds):
    ret = []
    path = []
    integerPartitionHelper(Sum,UpperBounds,path,ret)
    return ret
    

def tensorProduct(atomicParameterNodeList):
    upperBounds = [len(x)-1 for x in atomicParameterNodeList]
    topDim = sum(upperBounds)
    faceLattice = dict()
    for i in range(topDim+1):
        partition = integerPartition(i,upperBounds)
        faces = set()
        for p in partition:
            #print(p)
            facelist = [atomicParameterNodeList[i][p[i]] for i in range(len(p))]
            faces |= set(product(*facelist))
        faceLattice[i] = faces
        
    return faceLattice 


def parameterNodeLoader(fileNameList):
    """For global parameter node"""
    pnlist = []
    for fname in fileNameList:
        with open(fname,'r') as file:
            currentParameterNode = json.load(file)
            recover = dict()
            for key in currentParameterNode:
                newkey = int(key)
                newvalue = []
                for face in currentParameterNode[key]:
                    tempFace = tuple([tuple(x) for x in face])
                    newvalue.append(tempFace)
                newvalue =set(newvalue)
                recover[newkey] = newvalue
            pnlist.append(recover)
            
    return pnlist

# need the dimension to be 1
def checkFace(faceCell,bodyCell):
    for i in range(len(faceCell)):
        if not set(faceCell[i]).issubset(set(bodyCell[i])):
            return False
    return True

def unionParameterNodeFaceLattice(pnlist):
    # left multiplication sparse matrix
    ret = []
    topDim = max(pnlist[0].keys())
    
    faceCells = dict()
    bodyCells = dict()
    for i in range(len(pnlist)):
        for vertex in pnlist[i][0]:
            if vertex not in bodyCells:
                bodyCells[vertex] = len(bodyCells)
                
    for d in range(1,topDim+1):
        diffMatrix = dict()
        faceCells = bodyCells
        bodyCells = dict()
        # generate d-dimensional faces set
        for i in range(len(pnlist)):
            for dFaces in pnlist[i][d]:
                if dFaces not in bodyCells:
                    bodyCells[dFaces] = len(bodyCells)
        
        for bodyCell in bodyCells:
            for faceCell in faceCells:
                if checkFace(faceCell,bodyCell):
                    row = bodyCells[bodyCell]
                    col = faceCells[faceCell]
                    diffMatrix[(row,col)] = 1
        
        ret.append(diffMatrix)
    
    return ret

In [11]:
filepathList = ['x_o_1.json','x_o_1.json', 'x_o_1.json'] 
apnFaceLattice = [loadAtomicChainComplex(file) for file in filepathList]
keysList = [x.keys() for x in apnFaceLattice]
#indexOfParameterNode = product(*keysList)
#firstParameterNode = next(indexOfParameterNode)
#firstParameterNode

In [12]:
fcList = []
for pn in pnlist:
    apnList = [apnFaceLattice[i][pn[i]] for i in range(len(pn))]

    # facelattice
    oneFaceLattice = tensorProduct(apnList)

    fcList.append(oneFaceLattice)

In [13]:
rnFaceLattice = dict()
for fl in fcList:
    for key in fl:
        if key in rnFaceLattice:
            rnFaceLattice[key] |= fl[key] 
        else:
            rnFaceLattice[key] = fl[key]

In [14]:
for d in rnFaceLattice:
    print(len(rnFaceLattice[d]))

216
864
1476
1368
726
210
26


In [15]:
ret = unionParameterNodeFaceLattice(fcList)

In [16]:
mats = [matrix(GF(2),r) for r in ret]
chains = ChainComplex(mats)
chains.betti()

{0: 1, 1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0}